In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import json

In [2]:
import pickle

def load_processed_data(file_path):
    with open(file_path, 'rb') as file:
        X, Y = pickle.load(file)
    return X, Y

In [3]:
X, Y = load_processed_data('res_processed_data.pkl')

In [4]:

X_filtered = [img for img in X if img is not None and isinstance(img, np.ndarray)]
Y_filtered = [Y[i] for i in range(len(Y)) if X[i] is not None and isinstance(X[i], np.ndarray)]

X_filtered = np.array(X_filtered)

Y_filtered = np.array(Y_filtered)
Y_filtered = Y_filtered[:, :2]

In [5]:
len(X_filtered), len(Y_filtered)

(3948, 3948)

In [6]:
Y_filtered.shape

(3948, 2)

In [7]:
import cv2
cv2.imshow('image', X_filtered[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X_filtered, Y_filtered, test_size=0.2, random_state=42)
#Val data
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

train_datagen = ImageDataGenerator(
    # rotation_range=20,
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # # shear_range=0.2,
    # zoom_range=[0.9, 1.1],
    # horizontal_flip=True,
    # fill_mode='nearest'
)

# Assuming you have your training data in train_data and train_labels
train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)


model = Sequential()

# Adding L2 Regularization to Convolutional Layers
l2_reg = 0.001

# First Conv Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 80, 3), kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D())
model.add(Dropout(0.1))

# Second Conv Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.15))

# Third Conv Block
model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# # Fourth Conv Block
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.2))

# # Fifth Conv Block
# model.add(Conv2D(512, (3, 3), activation='relu'))
# model.add(MaxPooling2D())
# model.add(Dropout(0.25))

# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2), activation ='sigmoid')  # Adjust the number of outputs as needed

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.00005), loss='mse', metrics=['mean_squared_error', 'mean_absolute_error'])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=100,  # Adjust number of epochs
    validation_data=(X_val, Y_val),  # Assuming validation data is available
      callbacks=[early_stopping],
    batch_size=16
)

Epoch 1/100
79/79 [==============================] - 11s 128ms/step - loss: 0.5603 - mean_squared_error: 0.4695 - mean_absolute_error: 0.5081 - val_loss: 0.4251 - val_mean_squared_error: 0.3345 - val_mean_absolute_error: 0.4799
Epoch 2/100
79/79 [==============================] - 10s 130ms/step - loss: 0.2584 - mean_squared_error: 0.1680 - mean_absolute_error: 0.3289 - val_loss: 0.3998 - val_mean_squared_error: 0.3096 - val_mean_absolute_error: 0.4591
Epoch 3/100
79/79 [==============================] - 10s 129ms/step - loss: 0.2271 - mean_squared_error: 0.1372 - mean_absolute_error: 0.2973 - val_loss: 0.3649 - val_mean_squared_error: 0.2754 - val_mean_absolute_error: 0.4291
Epoch 4/100
79/79 [==============================] - 10s 129ms/step - loss: 0.2094 - mean_squared_error: 0.1202 - mean_absolute_error: 0.2762 - val_loss: 0.3245 - val_mean_squared_error: 0.2355 - val_mean_absolute_error: 0.3942
Epoch 5/100
79/79 [==============================] - 10s 130ms/step - loss: 0.2060 - mea

In [10]:
#evaluate the model
model.evaluate(X_test, Y_test)

25/25 [==============================] - 1s 23ms/step - loss: 0.0542 - mean_squared_error: 0.0252 - mean_absolute_error: 0.1165


[0.05416402593255043, 0.025179727002978325, 0.11654376238584518]

In [34]:
from keras.models import load_model
model = load_model('./models/eye_gaze_v13.h5')

In [14]:
#plot predicted vs actual on test data on a canvas using opencv 

import cv2
import numpy as np 
predictions = model.predict(X_test)
screen_width, screen_height = 2650, 1440
canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
cv2.namedWindow('Gaze Tracking on Canvas', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Gaze Tracking on Canvas', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# plot the first 10 images one by one
for i in range(0,55):
    
    # get the predicted x,y coordinates
    x, y = predictions[i][0] * screen_width, predictions[i][1] * screen_height

    # lock the preds 
    x = min(max(x, 0), screen_width)
    y = min(max(y, 0), screen_height)

    # get the actual x,y coordinates
    x_actual, y_actual = Y_test[i][0] * screen_width, Y_test[i][1] * screen_height

    # plot the predicted x,y coordinates
    cv2.circle(canvas, (int(x), int(y)), 10, (0, 0, 255), -1)
    # plot the actual x,y coordinates
    cv2.circle(canvas, (int(x_actual), int(y_actual)), 10, (0, 255, 0), -1)
    # show the canvas
    cv2.imshow('Gaze Tracking on Canvas', canvas)
    cv2.waitKey(0)
    cv2.imshow('Gaze Tracking on Canvas', X_test[i])
    cv2.waitKey(0)
    # clear the canvas
    
    canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)

cv2.waitKey(0)
cv2.destroyAllWindows()

25/25 [==============================] - 1s 20ms/step


In [15]:
model.save('./models/eye_gaze_v13.h5')